In [ ]:
import numpy 
from tensorflow import keras
from keras.constraints import maxnorm
#install package from conda-forge
from sklearn.preprocessing import OneHotEncoder
from keras.models import Sequential , load_model
from keras.layers import Conv2D , MaxPool2D , Dense , Dropout , Flatten
from keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix
from keras.utils import np_utils
from keras.datasets import cifar10
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
#load images from cifar10 dataset
# x : original image
# y : label of image
(x_train , y_train) , (x_test , y_test) = cifar10.load_data()

In [ ]:
num_class = y_test.shape[1] #10 
# x_train.shape
# (5000 , 32 ,32 , 3) 32 px * 32 px and 3 for RGB channel
# x_test.shape
# show image :
# plt.imshow(x_train[0])


In [ ]:
# normalize the input from 0 - 255 to 0  - 1 by dividing by 255
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train = x_train / 255.0

x_test = x_test / 255.0

In [ ]:
#one-hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

#before this : y_train[0] : [6]
#after this : y_train[0] : [0 , 0, 0, 0, ,0 , 0 ,1 , 0  , 0 ,0 ]

In [ ]:
#building model
model = Sequential()
model.add(Conv2D(32 , (3 , 3) , padding='same' ,input_shape = x_train.shape[1:], activation='relu'))
model.add(Conv2D(64 , (3 , 3) , padding='same' , activation='relu'))
model.add(MaxPool2D())
model.add(Conv2D(64 , (3 , 3) , padding='same' , activation='relu'))
model.add(MaxPool2D())
model.add(Conv2D(128 , (3 , 3) , padding='same' , activation='relu'))
model.add(MaxPool2D())
# flatten means convert nD array to 1D array
model.add(Flatten())



In [ ]:
#input layer for neural network :
model.add(Dense(32 , activation='relu'))
#output layer :
model.add(Dense(10 , activation='softmax'))

In [ ]:
#compile model :
#if we have more than 2 categories  :
model.compile(loss='categorical_crossentropy' , optimizer='Adam' , metrics=['accuracy' ])

#else : loss='binary_crossentropy'

model.summary()

In [ ]:
#training the model:
history = model.fit(x_train , y_train , validation_data=(x_test , y_test) , batch_size=64,epochs=30 )

In [ ]:
#show loss , accuracy , val_loss , val_accuracy
pd.DataFrame(history.history).plot()
#overfitting :
#loss is decreasing but val_loss in increase